PRESENTATION DU JEU DE DONNEES : 

- Source et origine des données : 
Le jeu de données est intitulé "Domiciliation des agents - Bilan social" et est publié sur la plateforme Open Data Paris.
Ces données proviennent du bilan social des agents de la Ville de Paris et portent sur leur lieu de résidence.

Nombre d'observations et de variables : 
- nombre de lignes : 237 688
- nombre de colonnes : 11

Types et significations des variables avant modification de la base de données : 
- "Date" : entier (int64), corresponds à l'année du bilan social.
- "Collective" : objet (object), designe la collectivité à laquelle les agents sont rattachés.
- "Sexe" : objet (object), designe les genres des agents (masculin, féminin).
- "Direction" : objet (object), corresponds à l'affectation de l'agent.
- "Catégorie" : objet (object), ce sont les status des agents (A, B, C).
- "Filière" : Object, le domaine de l'agent aka son service.
- "Zone" : Object, il s'agit de la zone géographique de domicialition.
- "Code" : Object, code postale de la ville de domiciliation. 
- "Ville" : Object, nom de la ville.
- "Agent" : Nombre décimal (float64), c'est le nombre d'agent dans cette combinaison de critères.
- "Coordonnees_xy_code_postal" : objet (object), ce sont les coordonnées géographiques (latitude, longitude) du code postal.

Nombre de valeurs manquantes par variable : 
Filière : 40 702
Collectivite : 36 278
Ville : 19 108
Direction : 13 896
Coordonnées : 2 795
Catégorie : 1 374
Code postal : 90
Les autres variables ne contiennent aucune valeur manquante.

Autres éléments descriptifs pertinents : 
- Le jeu de données est agrégé par année, collectivité, les caractéristique de l'agent, le lieu de domiciliation, le nombre d'agent. 
- Historique : on voit une périodicité avec les années, ce qui permet des analyses temporelles. 
- Les adresses : les colonnes "code postal" et "coordonnees_xy_code_postal" nécessite une attention particulière pour le nettoyage et la conversion en types numériques.

IMPORTATION DES LIBRAIRIES

In [35]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

CHARGEMENT DU FICHIER CSV

In [36]:
file_path = "domiciliation-des-agents-bilan-social.csv"

#Chargement de DataFrame : nous vérifions que la base de données ait bien été importée.
try : 
    df = pd.read_csv(file_path, sep=';')
    print(f"Le fichier '{file_path}' a été chargé avec succès")
except FileNotFoundError :
    print(f"Le fichier '{file_path}' est introuvable. Vérifier le dossier")
    df = pd.DataFrame()

Le fichier 'domiciliation-des-agents-bilan-social.csv' a été chargé avec succès


STRUCTURE ET DESCRIPTION DU JEU DE DONNEES : 
Les types de données manquantes : On vérifie les types de données (`dtypes`) et on calcule le nombre de valeurs manquantes (`NaN`) par colonne.

Types de données

In [37]:
# Afficher le nombre de lignes et de colonnes 
nombre_lignes, nombre_colonnes = df.shape
print(f"Nombre de lignes : {nombre_lignes}")
print(f"Nombre de colonnes : {nombre_colonnes}")

#Afficher les types de données de chaque colonne
print("Types de données par variable:")
print(df.dtypes)

Nombre de lignes : 237688
Nombre de colonnes : 11
Types de données par variable:
DATE                            int64
COLLECTIVITE                   object
SEXE                           object
DIRECTION                      object
CATEGORIE                      object
FILIERE                        object
ZONE                           object
CODE POSTAL                    object
VILLE                          object
AGENT                         float64
coordonnees_xy_code_postal     object
dtype: object


Données manquantes

In [38]:
# Calcul du nombre de valeurs manquantes par variable
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_summary = pd.DataFrame({
    'Valeurs Manquantes (N)': missing_values,
    'Pourcentage Manquant (%)': missing_percentage.round(2)
})
print("--- Tableau des valeurs manquantes par variable ---")
print(missing_summary[missing_summary['Valeurs Manquantes (N)'] > 0].sort_values(by='Pourcentage Manquant (%)', ascending=False).to_markdown(numalign="left", stralign="left"))

--- Tableau des valeurs manquantes par variable ---
|                            | Valeurs Manquantes (N)   | Pourcentage Manquant (%)   |
|:---------------------------|:-------------------------|:---------------------------|
| FILIERE                    | 40702                    | 17.12                      |
| COLLECTIVITE               | 36278                    | 15.26                      |
| VILLE                      | 19108                    | 8.04                       |
| DIRECTION                  | 13896                    | 5.85                       |
| coordonnees_xy_code_postal | 2795                     | 1.18                       |
| CATEGORIE                  | 1374                     | 0.58                       |
| CODE POSTAL                | 90                       | 0.04                       |


CONVERSION ET NETTOYAGE DES DONNEES POUR UNE MEILLEURE PERFORMANCEET CREATION DE NOUVELLES VARIABLES

Conversion d'une variable type float en entier. Mais avant cela, nous Vérifions que la colonne "AGENT" contient des valeurs non entières.

In [39]:
# On vérifie si le modulo 1 de la valeur est différent de 0, ce qui indique une partie décimale
decimal_agents = df[df['AGENT'].notnull() & (df['AGENT'] % 1 != 0)]

if not decimal_agents.empty:
    print("\n--- Statistiques de la colonne 'AGENT' (Contient des valeurs décimales) ---")
    print(df['AGENT'].describe().to_markdown(numalign="left", stralign="left"))
    # Affichage de quelques exemples de valeurs décimales
    print("\nQuelques exemples de valeurs décimales dans 'AGENT' :")
    print(decimal_agents['AGENT'].head().to_markdown(numalign="left", stralign="left"))
else:
    print("\nLa colonne 'AGENT' ne contient pas de valeurs décimales (sauf NaN, déjà exclues). Conversion en 'int64' possible.")
    # Puisque AGENT n'a pas de NaNs, on peut convertir
    df['AGENT'] = df['AGENT'].astype('int64')
    print("Colonne 'AGENT' convertie en `int64`.")
    print("\n--- Statistiques de la colonne 'AGENT' (Après conversion) ---")
    print(df['AGENT'].describe().to_markdown(numalign="left", stralign="left"))


La colonne 'AGENT' ne contient pas de valeurs décimales (sauf NaN, déjà exclues). Conversion en 'int64' possible.
Colonne 'AGENT' convertie en `int64`.

--- Statistiques de la colonne 'AGENT' (Après conversion) ---
|       | AGENT   |
|:------|:--------|
| count | 237688  |
| mean  | 2.5549  |
| std   | 8.0234  |
| min   | 1       |
| 25%   | 1       |
| 50%   | 1       |
| 75%   | 2       |
| max   | 448     |


Explication et Interprétation des valeurs ici : 
- Count représente le nombre total d'observations (lignes) dans notre jeu de données. 
Cela confirme qu'il n'y a aucune valeur manquante dans cette colonne.
- Mean corresponds à la moyenne du nombre d'agents par ligne. En moyenne, chaque ligne d'entrée (chaque combinaison unique de date, sexe, catégorie, ville, etc.) représente environ 2.55 agents.
- Std mesure de la dispersion des données autour de la moyenne. Un écart-type élevé (8.02) par rapport à la moyenne (2.55) indique que le nombre d'agents par ligne est très variable. Certaines lignes représentent beaucoup d'agents, et d'autres très peu.
- Min corresponds à la valeur minimale. La plus petite agrégation possible est 1 agent, ce qui est logique pour un comptage d'effectifs.
- Le premier quartile des lignes du DataFrame représentent 1 agent.
- La médiane corresponds à des lignes du DataFrame (la moitié) représentent 1 agent. Le fait que la médiane soit égale au minimum indique que la grande majorité des lignes sont très granulaires.
- Le troisième quartile corresponds à des lignes du DataFrame représentent 2 agents ou moins. Inversement, seulement 25% des lignes représentent 3 agents ou plus.
- Max représente la ligne la plus agrégée de notre jeu de données (448 agents). 

Gestion des valeurs manquantes dans les colonnes catégorielles : 
Nous remplacerons les valeurs manquantes "NaN" par "Non renseigné" dans tous les champs. Dans un premier temps, nous le ferons pour les variables types "object". 

In [40]:
categorical_cols_to_fill = ['COLLECTIVITE', 'DIRECTION', 'CATEGORIE', 'FILIERE']
df[categorical_cols_to_fill] = df[categorical_cols_to_fill].fillna('Non renseigné')
print(df)

        DATE    COLLECTIVITE      SEXE DIRECTION CATEGORIE            FILIERE  \
0       2014   Non renseigné  MASCULIN       DPE         C      Non renseigné   
1       2014   Non renseigné  MASCULIN       DPA         C      Non renseigné   
2       2014   Non renseigné   FEMININ       DAS         C      Non renseigné   
3       2014   Non renseigné   FEMININ     DASES         B      Non renseigné   
4       2014   Non renseigné   FEMININ      DFPE         C      Non renseigné   
...      ...             ...       ...       ...       ...                ...   
237683  2022  VILLE DE PARIS  MASCULIN       DPE         C  FILIERE TECHNIQUE   
237684  2022  VILLE DE PARIS  MASCULIN       DPE         C  FILIERE TECHNIQUE   
237685  2022  VILLE DE PARIS  MASCULIN       DPE         C  FILIERE TECHNIQUE   
237686  2022  VILLE DE PARIS  MASCULIN       DPE         C  FILIERE TECHNIQUE   
237687  2022  VILLE DE PARIS  MASCULIN       DPE         C  FILIERE TECHNIQUE   

                   ZONE COD

Nous allons transformer la colonne "coordonnees_xy_code_postal" afin de l'optimiser et utiliser les différents données insérés, notamment la longitude et la latitude. 
On fini par supprimer cette colonne qui nous ai plus utile et ainsi nous gagnons de l'espace. On passe de 11 colonnes à 12 colonnes.

In [41]:
# Séparation de la chaîne de coordonnées en deux colonnes
coords = df['coordonnees_xy_code_postal'].str.split(', ', expand=True)

# Création des nouvelles colonnes LATITUDE et LONGITUDE et conversion en float
# errors='coerce' transforme les valeurs non convertibles (incluant les NaNs) en NaN
df['LATITUDE'] = pd.to_numeric(coords[0], errors='coerce')
df['LONGITUDE'] = pd.to_numeric(coords[1], errors='coerce')

# Suppression de la colonne d'origine des coordonnées
df = df.drop(columns=['coordonnees_xy_code_postal'])

print("--- État du DataFrame après nettoyage et conversion ---")
df.info()

--- État du DataFrame après nettoyage et conversion ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237688 entries, 0 to 237687
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   DATE          237688 non-null  int64  
 1   COLLECTIVITE  237688 non-null  object 
 2   SEXE          237688 non-null  object 
 3   DIRECTION     237688 non-null  object 
 4   CATEGORIE     237688 non-null  object 
 5   FILIERE       237688 non-null  object 
 6   ZONE          237688 non-null  object 
 7   CODE POSTAL   237598 non-null  object 
 8   VILLE         218580 non-null  object 
 9   AGENT         237688 non-null  int64  
 10  LATITUDE      234893 non-null  float64
 11  LONGITUDE     234893 non-null  float64
dtypes: float64(2), int64(2), object(8)
memory usage: 21.8+ MB


In [43]:
# Nous affichons le résultat des 10 premières lignes pour voir l'affichage et l'exécution de notre demande.
print("\n--- Premières 10 lignes du DataFrame après nettoyage et conversion ---")
print(df.head(10).to_markdown(index=False, numalign="left", stralign="left"))


--- Premières 10 lignes du DataFrame après nettoyage et conversion ---
| DATE   | COLLECTIVITE   | SEXE     | DIRECTION   | CATEGORIE   | FILIERE       | ZONE            | CODE POSTAL   | VILLE                     | AGENT   | LATITUDE   | LONGITUDE   |
|:-------|:---------------|:---------|:------------|:------------|:--------------|:----------------|:--------------|:--------------------------|:--------|:-----------|:------------|
| 2014   | Non renseigné  | MASCULIN | DPE         | C           | Non renseigné | GRANDE COURONNE | 77720         | GRANDPUITS BAILLY CARROIS | 1       | 48.585     | 2.8965      |
| 2014   | Non renseigné  | MASCULIN | DPA         | C           | Non renseigné | GRANDE COURONNE | 91310         | LINAS                     | 1       | 48.6317    | 2.2665      |
| 2014   | Non renseigné  | FEMININ  | DAS         | C           | Non renseigné | PROVINCE        | 2600          | VILLIERS COTTERETS        | 1       | nan        | nan         |
| 2014   | Non ren

In [44]:
# --- Valeurs manquantes avant imputation contextuelle ---
initial_nan_ville = df['VILLE'].isnull().sum()
initial_nan_cp = df['CODE POSTAL'].isnull().sum()
print("--- Valeurs manquantes avant imputation contextuelle ---")
print(f"VILLE manquantes : {initial_nan_ville}")
print(f"CODE POSTAL manquants : {initial_nan_cp}")

--- Valeurs manquantes avant imputation contextuelle ---
VILLE manquantes : 19108
CODE POSTAL manquants : 90


Imputation des colonnes "Code postal" et "Ville" en fonction de ses valeurs respectives.
Dans un premier temps nous allons créer des mappings pour les colonnes : "code postal" et "ville".

In [45]:
# Création des mappings
# Map CP -> VILLE : Mode de VILLE pour chaque CODE POSTAL
cp_to_ville_map = df.groupby('CODE POSTAL')['VILLE'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan).dropna()
# Map VILLE -> CP : Mode de CODE POSTAL pour chaque VILLE
ville_to_cp_map = df.groupby('VILLE')['CODE POSTAL'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan).dropna()

Nous commençons l'imputation des données en fonction de celle qui existe déjà.

In [46]:
# Imputation de VILLE en fonction de CODE POSTAL. On mappe la VILLE uniquement si VILLE est NaN et CODE POSTAL est connu.
mask_ville_nan = df['VILLE'].isnull() & df['CODE POSTAL'].notnull()
df.loc[mask_ville_nan, 'VILLE'] = df.loc[mask_ville_nan, 'CODE POSTAL'].map(cp_to_ville_map)

# Imputation de CODE POSTAL en fonction de VILLE. On mappe le CODE POSTAL uniquement si CODE POSTAL est NaN et VILLE est connue.
mask_cp_nan = df['CODE POSTAL'].isnull() & df['VILLE'].notnull()
df.loc[mask_cp_nan, 'CODE POSTAL'] = df.loc[mask_cp_nan, 'VILLE'].map(ville_to_cp_map)

In [47]:
print("--- Imputation VILLE/CP en fonction des coordonnées ---")
coords_to_ville_map = df.groupby(['LATITUDE', 'LONGITUDE'])['VILLE'].agg(lambda x: x.mode()[0] if not x.mode().empty and x.mode()[0] is not None else np.nan)
coords_to_ville_map = coords_to_ville_map.dropna()
coords_to_cp_map = df.groupby(['LATITUDE', 'LONGITUDE'])['CODE POSTAL'].agg(lambda x: x.mode()[0] if not x.mode().empty and x.mode()[0] is not None else np.nan)
coords_to_cp_map = coords_to_cp_map.dropna()
mask_ville_nan_remaining = df['VILLE'].isnull() & df['LATITUDE'].notnull()
df.loc[mask_ville_nan_remaining, 'VILLE'] = df.loc[mask_ville_nan_remaining, ['LATITUDE', 'LONGITUDE']].apply(
    lambda row: coords_to_ville_map.get(tuple(row)), axis=1
)
mask_cp_nan_remaining = df['CODE POSTAL'].isnull() & df['LATITUDE'].notnull()
df.loc[mask_cp_nan_remaining, 'CODE POSTAL'] = df.loc[mask_cp_nan_remaining, ['LATITUDE', 'LONGITUDE']].apply(
    lambda row: coords_to_cp_map.get(tuple(row)), axis=1
)

--- Imputation VILLE/CP en fonction des coordonnées ---


In [48]:
# Bilan et Finalisation
print("\n--- Bilan des valeurs manquantes après toutes les imputations ---")
nan_ville_remaining = df['VILLE'].isnull().sum()
nan_cp_remaining = df['CODE POSTAL'].isnull().sum()

print(f"VILLE manquantes restantes : {nan_ville_remaining}")
print(f"CODE POSTAL manquants restants : {nan_cp_remaining}")

# Finalisation : Remplir les NaNs restants (ceux qui manquaient toujours)
df['VILLE'] = df['VILLE'].fillna('Non renseigné')
df['CODE POSTAL'] = df['CODE POSTAL'].fillna('Non renseigné')

# Vérification finale
final_nan_ville = df['VILLE'].isnull().sum()
final_nan_cp = df['CODE POSTAL'].isnull().sum()

print(f"\nFinal VILLE manquantes : {final_nan_ville}")
print(f"Final CODE POSTAL manquants : {final_nan_cp}")

# Affichage d'un échantillon des lignes VILLE/CP imputées par coordonnées
# On peut chercher les lignes imputées dans la première étape où VILLE était NaN, mais coordonnée connue
print("\n--- Exemples de lignes VILLE/CP complétées (après imputation) ---")
# Affichage des 5 lignes où la latitude est connue mais la ville était initialement manquante (on cherche dans l'original)
# Pour une vérification simple, on affiche les premières lignes de la base qui ont maintenant des valeurs
print(df[df['VILLE'] != 'Non renseigné'][['CODE POSTAL', 'VILLE', 'LATITUDE', 'LONGITUDE']].head(5).to_markdown(index=False, numalign="left", stralign="left"))


--- Bilan des valeurs manquantes après toutes les imputations ---
VILLE manquantes restantes : 161
CODE POSTAL manquants restants : 79

Final VILLE manquantes : 0
Final CODE POSTAL manquants : 0

--- Exemples de lignes VILLE/CP complétées (après imputation) ---
| CODE POSTAL   | VILLE                     | LATITUDE   | LONGITUDE   |
|:--------------|:--------------------------|:-----------|:------------|
| 77720         | GRANDPUITS BAILLY CARROIS | 48.585     | 2.8965      |
| 91310         | LINAS                     | 48.6317    | 2.2665      |
| 2600          | VILLIERS COTTERETS        | nan        | nan         |
| 91600         | SAVIGNY SUR ORGE          | 48.684     | 2.34918     |
| 92260         | FONTENAY AUX ROSES        | 48.7896    | 2.28707     |


Nous complétons les données des colonnes "Longitude" et "Latitude" coordonnées géographique.

In [49]:
initial_missing_coords = df[['LATITUDE', 'LONGITUDE']].isnull().sum().sum()
print(f"--- Bilan avant imputation finale des coordonnées ---")
print(f"Nombre total de valeurs manquantes dans LATITUDE/LONGITUDE : {initial_missing_coords}")
print("\n--- Imputation par médiane de CODE POSTAL ---")
def impute_coords(series):
    # Calcule la médiane uniquement sur les valeurs non NaN
    median_val = series.median()
    # Impute les NaNs avec la médiane du groupe
    return series.fillna(median_val)
df['LATITUDE'] = df.groupby('CODE POSTAL')['LATITUDE'].transform(impute_coords)
df['LONGITUDE'] = df.groupby('CODE POSTAL')['LONGITUDE'].transform(impute_coords)

--- Bilan avant imputation finale des coordonnées ---
Nombre total de valeurs manquantes dans LATITUDE/LONGITUDE : 5590

--- Imputation par médiane de CODE POSTAL ---


C:\Users\PVeena\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\PVeena\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\PVeena\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\PVeena\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\PVeena\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\PVeena\AppData\Roamin

Nous vérifions les valeurs manquantes et complétons le tous pour ne plus en avoir.

In [50]:
# Complémentation des Reliquats avec la médiane globale (Fallback)
remaining_missing_coords = df[['LATITUDE', 'LONGITUDE']].isnull().sum().sum()
print(f"Nombre de valeurs manquantes restantes après imputation par CODE POSTAL : {remaining_missing_coords}")

if remaining_missing_coords > 0:
    median_latitude_global = df['LATITUDE'].median()
    median_longitude_global = df['LONGITUDE'].median()

    # Imputation des NaNs restants avec la médiane globale
    df['LATITUDE'] = df['LATITUDE'].fillna(median_latitude_global)
    df['LONGITUDE'] = df['LONGITUDE'].fillna(median_longitude_global)
    print(f"Complétées avec la médiane globale (Lat: {median_latitude_global:.4f}, Lon: {median_longitude_global:.4f}).")

# Vérification finale
final_missing_coords = df[['LATITUDE', 'LONGITUDE']].isnull().sum().sum()
print(f"\nNombre total de valeurs manquantes FINAL dans les coordonnées : {final_missing_coords}")

# Affichage des types finaux pour confirmation
print("\n--- Aperçu des types de données finaux ---")
df.info()

Nombre de valeurs manquantes restantes après imputation par CODE POSTAL : 5568
Complétées avec la médiane globale (Lat: 48.8444, Lon: 2.3799).

Nombre total de valeurs manquantes FINAL dans les coordonnées : 0

--- Aperçu des types de données finaux ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237688 entries, 0 to 237687
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   DATE          237688 non-null  int64  
 1   COLLECTIVITE  237688 non-null  object 
 2   SEXE          237688 non-null  object 
 3   DIRECTION     237688 non-null  object 
 4   CATEGORIE     237688 non-null  object 
 5   FILIERE       237688 non-null  object 
 6   ZONE          237688 non-null  object 
 7   CODE POSTAL   237688 non-null  object 
 8   VILLE         237688 non-null  object 
 9   AGENT         237688 non-null  int64  
 10  LATITUDE      237688 non-null  float64
 11  LONGITUDE     237688 non-null  float64
dtypes: float64(2),

In [52]:
print("\n--- Exemple de lignes complétées (toutes colonnes) ---")
# Affichage d'une ligne où les coordonnées étaient manquantes (comme celle avec CODE POSTAL '75009')
example_imputed = df[df['CODE POSTAL'] == '75009'].head(1)
print(example_imputed[['CODE POSTAL', 'VILLE', 'LATITUDE', 'LONGITUDE']].to_markdown(index=False, numalign="left", stralign="left"))


--- Exemple de lignes complétées (toutes colonnes) ---
| CODE POSTAL   | VILLE    | LATITUDE   | LONGITUDE   |
|:--------------|:---------|:-----------|:------------|
| 75009         | PARIS 09 | 48.8769    | 2.33737     |


In [53]:
# Affichage des 10 premières lignes du DataFrame final
print("--- Les 10 premières lignes du DataFrame après nettoyage et complétion ---")
print(df.head(10).to_markdown(index=False, numalign="left", stralign="left"))

--- Les 10 premières lignes du DataFrame après nettoyage et complétion ---
| DATE   | COLLECTIVITE   | SEXE     | DIRECTION   | CATEGORIE   | FILIERE       | ZONE            | CODE POSTAL   | VILLE                     | AGENT   | LATITUDE   | LONGITUDE   |
|:-------|:---------------|:---------|:------------|:------------|:--------------|:----------------|:--------------|:--------------------------|:--------|:-----------|:------------|
| 2014   | Non renseigné  | MASCULIN | DPE         | C           | Non renseigné | GRANDE COURONNE | 77720         | GRANDPUITS BAILLY CARROIS | 1       | 48.585     | 2.8965      |
| 2014   | Non renseigné  | MASCULIN | DPA         | C           | Non renseigné | GRANDE COURONNE | 91310         | LINAS                     | 1       | 48.6317    | 2.2665      |
| 2014   | Non renseigné  | FEMININ  | DAS         | C           | Non renseigné | PROVINCE        | 2600          | VILLIERS COTTERETS        | 1       | 48.8444    | 2.37986     |
| 2014   | Non 

Nous allons traiter les doublons afin de nettoyer davantage notre jeu de données.

In [54]:
initial_rows = len(df)
# Compte le nombre de lignes strictement identiques (toutes colonnes confondues)
duplicate_count = df.duplicated().sum()
print (duplicate_count)


12


In [55]:
# On supprime les doublons
df = df.drop_duplicates()
rows_after_duplicates = len(df)


Nous traitons les incohérences dans notre jeu de données comme la standardisation de la casse en majuscule, la suppression des éspaces inutiles en début et fin de données et le remplacement des espaces multiples par un seul '\s+'.

In [56]:
object_cols = df.select_dtypes(include='object').columns
df_initial_for_comparison = pd.read_csv('domiciliation-des-agents-bilan-social.csv', sep=';')
unique_counts_before = df_initial_for_comparison[object_cols].nunique()

for col in object_cols:
    # 1. Mise en majuscules (standardisation de la casse)
    df[col] = df[col].astype(str).str.upper()
    # 2. Suppression des espaces inutiles (début/fin)
    df[col] = df[col].str.strip()
    # 3. Remplacement des espaces multiples par un seul
    df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
    
unique_counts_after = df[object_cols].nunique()
comparison_df = pd.DataFrame({'Unique Avant Normalisation (Initial)': unique_counts_before,
                              'Unique Après Normalisation (Final)': unique_counts_after}).dropna().astype(int)


Nous vérifions si notre nettoyage et conversion de données a porté ses fruits.

In [59]:
print(f"--- Bilan du Traitement des Doublons ---")
print(f"Nombre de lignes initiales : {initial_rows}")
print(f"Nombre de doublons trouvés et supprimés : {duplicate_count}")
print(f"Nombre de lignes après suppression des doublons : {rows_after_duplicates}")

print(f"\n--- Bilan du Traitement des Incohérences (Normalisation des chaînes) ---")
print("Effet de la normalisation sur le nombre de valeurs uniques :")
print(comparison_df.to_markdown(numalign='left', stralign='left'))

print("\n--- Aperçu des 10 premières lignes après nettoyage final ---")
print(df.head(10).to_markdown(index=False, numalign="left", stralign="left"))

--- Bilan du Traitement des Doublons ---
Nombre de lignes initiales : 237688
Nombre de doublons trouvés et supprimés : 12
Nombre de lignes après suppression des doublons : 237676

--- Bilan du Traitement des Incohérences (Normalisation des chaînes) ---
Effet de la normalisation sur le nombre de valeurs uniques :
|              | Unique Avant Normalisation (Initial)   | Unique Après Normalisation (Final)   |
|:-------------|:---------------------------------------|:-------------------------------------|
| COLLECTIVITE | 3                                      | 4                                    |
| SEXE         | 2                                      | 2                                    |
| DIRECTION    | 39                                     | 40                                   |
| CATEGORIE    | 3                                      | 4                                    |
| FILIERE      | 16                                     | 17                                   |
| ZONE 

Nous allons créer une nouvelle variable pour simplifier la visibilité des différentes villes aka "Zone" où les agents sont domiciliés. 

In [60]:
# Utilise np.where pour affecter 'PARIS' si la colonne ZONE contient 'PARIS', et 'HORS PARIS' sinon.
df['ZONE_SIMPLIFIEE'] = np.where(df['ZONE'].str.contains('PARIS'), 'PARIS', 'HORS PARIS')

# Vérification des effectifs dans la nouvelle colonne
print("--- Répartition des effectifs par ZONE_SIMPLIFIEE ---")
# Utilisation de AGENT comme poids pour calculer la somme des effectifs par nouvelle zone
repartition_agents = df.groupby('ZONE_SIMPLIFIEE')['AGENT'].sum()
print(repartition_agents.to_markdown(numalign="left", stralign="left"))

--- Répartition des effectifs par ZONE_SIMPLIFIEE ---
| ZONE_SIMPLIFIEE   | AGENT   |
|:------------------|:--------|
| HORS PARIS        | 378736  |
| PARIS             | 228522  |


Nous allons créer une nouvelle variable "Niveau de qualification" en utilisant la colonne "Catégorie" afin d'avoir un détail supplémentaire.

In [61]:
# Utilisation d'un dictionnaire pour mapper les catégories A, B, C
qualification_mapping = {
    'A': 'CADRE / SUPÉRIEUR',
    'B': 'INTERMÉDIAIRE',
    'C': 'EXÉCUTION'
}

# La colonne CATEGORIE est déjà en majuscules et 'NON RENSEIGNÉ'
# Appliquer le mapping, et utiliser CATEGORIE elle-même comme fallback pour 'NON RENSEIGNÉ'
df['NIVEAU_QUALIFICATION'] = df['CATEGORIE'].map(qualification_mapping).fillna(df['CATEGORIE'])

# Vérification et analyse de la nouvelle colonne
print("--- Répartition des AGENTS par NIVEAU_QUALIFICATION ---")

# Calcul de la somme des agents (effectifs) pour chaque nouveau niveau
repartition_qualification = df.groupby('NIVEAU_QUALIFICATION')['AGENT'].sum().sort_values(ascending=False)
total_agents = repartition_qualification.sum()
repartition_pourcentage = (repartition_qualification / total_agents) * 100

summary_df = pd.DataFrame({
    'Effectif Total': repartition_qualification,
    'Pourcentage (%)': repartition_pourcentage.round(2)
})

print(summary_df.to_markdown(numalign="left", stralign="left"))

--- Répartition des AGENTS par NIVEAU_QUALIFICATION ---
| NIVEAU_QUALIFICATION   | Effectif Total   | Pourcentage (%)   |
|:-----------------------|:-----------------|:------------------|
| EXÉCUTION              | 399545           | 65.79             |
| INTERMÉDIAIRE          | 114705           | 18.89             |
| CADRE / SUPÉRIEUR      | 91164            | 15.01             |
| NON RENSEIGNÉ          | 1844             | 0.3               |


VERIFICATION DE L'EXISTENCE DES COLONNES CREEES.

In [62]:
# Nous vérifions l'existence des colonnes créées avant de les afficher
if 'ZONE_SIMPLIFIEE' not in df.columns:
    print("La colonne ZONE_SIMPLIFIEE doit être recréée pour l'affichage et l'export.")
    
    # Recréation de ZONE_SIMPLIFIEE
    df['ZONE_SIMPLIFIEE'] = np.where(df['ZONE'].str.contains('PARIS'), 'PARIS', 'HORS PARIS')

if 'NIVEAU_QUALIFICATION' not in df.columns:
    print("La colonne NIVEAU_QUALIFICATION doit être recréée pour l'affichage et l'export.")
    
    # Recréation de NIVEAU_QUALIFICATION
    qualification_mapping = {'A': 'CADRE / SUPÉRIEUR', 'B': 'INTERMÉDIAIRE', 'C': 'EXÉCUTION'}
    df['NIVEAU_QUALIFICATION'] = df['CATEGORIE'].map(qualification_mapping).fillna(df['CATEGORIE'])


# --- Affichage du DataFrame Final ---
print("--- Aperçu des 10 premières lignes du DataFrame final (avec les nouvelles variables) ---")
print(df.head(10).to_markdown(index=False, numalign="left", stralign="left"))

--- Aperçu des 10 premières lignes du DataFrame final (avec les nouvelles variables) ---
| DATE   | COLLECTIVITE   | SEXE     | DIRECTION   | CATEGORIE   | FILIERE       | ZONE            | CODE POSTAL   | VILLE                     | AGENT   | LATITUDE   | LONGITUDE   | ZONE_SIMPLIFIEE   | NIVEAU_QUALIFICATION   |
|:-------|:---------------|:---------|:------------|:------------|:--------------|:----------------|:--------------|:--------------------------|:--------|:-----------|:------------|:------------------|:-----------------------|
| 2014   | NON RENSEIGNÉ  | MASCULIN | DPE         | C           | NON RENSEIGNÉ | GRANDE COURONNE | 77720         | GRANDPUITS BAILLY CARROIS | 1       | 48.585     | 2.8965      | HORS PARIS        | EXÉCUTION              |
| 2014   | NON RENSEIGNÉ  | MASCULIN | DPA         | C           | NON RENSEIGNÉ | GRANDE COURONNE | 91310         | LINAS                     | 1       | 48.6317    | 2.2665      | HORS PARIS        | EXÉCUTION              |
| 2

Nous allons réorganiser notre DataFrame.

In [63]:
# Définition de l'ordre logique des colonnes
ordered_cols = [
    'DATE',
    'COLLECTIVITE',
    'DIRECTION',
    'CATEGORIE',
    'NIVEAU_QUALIFICATION',
    'FILIERE',
    'SEXE',
    'ZONE',
    'ZONE_SIMPLIFIEE',
    'CODE POSTAL',
    'VILLE',
    'LATITUDE',
    'LONGITUDE',
    'AGENT'
]

# Réorganisation du DataFrame
df = df[ordered_cols]

print("--- Les 10 premières lignes du DataFrame avec les colonnes réorganisées ---")
print(df.head(10).to_markdown(index=False, numalign="left", stralign="left"))

--- Les 10 premières lignes du DataFrame avec les colonnes réorganisées ---
| DATE   | COLLECTIVITE   | DIRECTION   | CATEGORIE   | NIVEAU_QUALIFICATION   | FILIERE       | SEXE     | ZONE            | ZONE_SIMPLIFIEE   | CODE POSTAL   | VILLE                     | LATITUDE   | LONGITUDE   | AGENT   |
|:-------|:---------------|:------------|:------------|:-----------------------|:--------------|:---------|:----------------|:------------------|:--------------|:--------------------------|:-----------|:------------|:--------|
| 2014   | NON RENSEIGNÉ  | DPE         | C           | EXÉCUTION              | NON RENSEIGNÉ | MASCULIN | GRANDE COURONNE | HORS PARIS        | 77720         | GRANDPUITS BAILLY CARROIS | 48.585     | 2.8965      | 1       |
| 2014   | NON RENSEIGNÉ  | DPA         | C           | EXÉCUTION              | NON RENSEIGNÉ | MASCULIN | GRANDE COURONNE | HORS PARIS        | 91310         | LINAS                     | 48.6317    | 2.2665      | 1       |
| 2014   | NON R

Nous voulons également pouvoir télécharger notre nouvelle base de données

In [64]:
# --- Création et Téléchargement du fichier CSV ---
# Définir le nom du fichier de sortie
output_filename = 'domiciliation_agents_nettoyee_et_enrichie.csv'

# Exporter le DataFrame au format CSV (sans l'index)
df.to_csv(output_filename, index=False, sep=';', encoding='utf-8')

print(f"\n--- Exportation réussie ---")
print(f"Le DataFrame nettoyé est prêt à être téléchargé sous le nom : {output_filename}")


--- Exportation réussie ---
Le DataFrame nettoyé est prêt à être téléchargé sous le nom : domiciliation_agents_nettoyee_et_enrichie.csv
